In [13]:
from __future__ import print_function
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import explode, col

In [5]:
def makePairs((user, ratings)):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))



SyntaxError: invalid syntax (<ipython-input-5-678c8718447f>, line 1)

In [ ]:
def filterDuplicates( (userID, ratings) ):
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2

In [ ]:
# Compute Similarity for each pair of items and predict the scores
# instead of passing rating pairs pass
def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    # change the loops - loop over all the rating pairs
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [15]:
if __name__ == "__main__":

    # Create a SparkSession (Note, the config section is only for Windows!)
    spark = SparkSession.builder.appName("RecommenderSystem").getOrCreate()

    
    
    # Load up data as dataframe
    ratingData = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/My_Data/MS/CS657/HW-4/InputData/ratings.csv")
    ratingData = ratingData.limit(500)
    ratingData.show(20)
    
    moviesData = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/My_Data/MS/CS657/HW-4/InputData/movies.csv")
    moviesData.limit(1000)
    #moviesData.show(20)
        
    ########################## ALS Model ############################################################################
    
#     train_data,test_data = ratingData.randomSplit([0.8,0.2])
    
#     recommender = ALS(maxIter = 5, nonnegative = True,implicitPrefs = False,coldStartStrategy="drop", 
#                       userCol='userId', itemCol='movieId', ratingCol='rating')
    
    
#     param_grid = ParamGridBuilder().addGrid(recommender.rank, [5, 10, 15]).addGrid(recommender.regParam, [.01]).build()
    
#     evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction") 
    
#     cv = CrossValidator(estimator=recommender, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
    
#     model = cv.fit(train_data)
#     model = model.bestModel
#     pred_data = model.transform(test_data)
#     pred_data.show()
    ########################################################################################################################
    
    
    ########################## RMSE , MSE and MAP for ALS  ####################################################################
    
#     RMSE = evaluator.evaluate(pred_data)
#     print("RMSE = %s" % RMSE)
#     MSE = evaluator.evaluate(pred_data, {evaluator.metricName: "mse"})
#     # Mean absolute error
#     print("MAE = %s" % MSE)
    
    ########################################################################################################################
    
    
    ########################## View reommendations in interpretable format for ALS  #################################################################
#     recommendations = model.recommendForAllUsers(5)
#     recommendations.show()
    
#     recomentdationFormatted = recommendations.withColumn("recommendFormat", explode("recommendations")).select('userId', col("recommendFormat.movieId"), col("recommendFormat.rating"))
#     recomentdationFormatted.limit(10).show()
    
#     recomentdationFormatted.join(moviesData, on='movieId').filter('userId = 1').show()
    
#     ratingData.join(moviesData, on='movieId').filter('userId = 1').sort('rating', ascending=False).limit(10).show()
    ########################################################################################################################
    
    
    ########################## Collaborative item-item  filtering ##############################################################
    
    # Map ratings to key / value pairs: user ID => movie ID, rating
    data = spark.textFile("C:/My_Data/MS/CS657/HW-4/InputData/ratings.csv")
    # print(data.collect())
    ratings = data.map(lambda l: l.split("::")).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

    print(ratings.collect())
    
    
    ########################################################################################################################
    

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
|     1|    112|   3.5|1094785740|
|     1|    151|   4.0|1094785734|
|     1|    223|   4.0|1112485573|
|     1|    253|   4.0|1112484940|
|     1|    260|   4.0|1112484826|
|     1|    293|   4.0|1112484703|
|     1|    296|   4.0|1112484767|
|     1|    318|   4.0|1112484798|
|     1|    337|   3.5|1094785709|
|     1|    367|   3.5|1112485980|
|     1|    541|   4.0|1112484603|
|     1|    589|   3.5|1112485557|
|     1|    593|   3.5|1112484661|
|     1|    653|   3.0|1094785691|
|     1|    919|   3.5|1094785621|
+------+-------+------+----------+
only showing top 20 rows



AttributeError: 'SparkSession' object has no attribute 'textFile'